In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import model_selection, metrics
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt
import seaborn as sns
import math
import keras
from time import time
import glob
from keras.callbacks import TensorBoard


import os
print(os.listdir("."))

/home/lazukav/anaconda3/lib/python3.5/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


['.ipynb_checkpoints', 'mnist_submit-Copy3.ipynb', 'model2.csv', 'test.csv', 'train.csv', 'Untitled1.ipynb', 'model0.csv', 'model2.ipynb', 'model1.csv', 'model1.ipynb', 'mnist_submit.ipynb', 'EDA.ipynb', 'Untitled.ipynb']


Using TensorFlow backend.


In [2]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

In [3]:
def load_data(train_path='train.csv', test_path='test.csv'):
    train = pd.DataFrame.from_csv(train_path, index_col=None)
    test = pd.DataFrame.from_csv(test_path, index_col=None)
              
    return train, test

In [4]:
train, test = load_data()

/home/lazukav/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  
/home/lazukav/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
train_X = train.drop("label", axis=1)
train_y = train["label"]

In [6]:
train_X = train_X.div(255, axis=0).fillna(0)
test = test.div(255, axis=0).fillna(0)

In [7]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(train_X, train_y, test_size=0.2, shuffle=None)

In [8]:
cols = [tf.feature_column.numeric_column(i) for i in train_X.columns.values]

In [9]:
class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='./logs', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input

In [11]:
batch_size = 128
num_classes = 10
epochs = 5 # 12
pool_size = 2
kernel_size = 3

In [12]:
X_train = X_train.values.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.values.reshape(X_test.shape[0], 28, 28, 1)

In [13]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [14]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [15]:
input_shape = (X_train.shape[1], X_train.shape[2], 1)

In [16]:
model = Sequential()
model.add(Conv2D(32, kernel_size=kernel_size,
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, kernel_size=kernel_size, activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=[TrainValTensorBoard(write_graph=False,log_dir="/tmp/model2")])
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 33600 samples, validate on 8400 samples
Epoch 1/5
33600/33600 [==============================] - 48s 1ms/step - loss: 0.3750 - acc: 0.8828 - val_loss: 0.1003 - val_acc: 0.9694
Epoch 2/5
33600/33600 [==============================] - 49s 1ms/step - loss: 0.1128 - acc: 0.9657 - val_loss: 0.0682 - val_acc: 0.9795
Epoch 3/5
33600/33600 [==============================] - 47s 1ms/step - loss: 0.0804 - acc: 0.9743 - val_loss: 0.0545 - val_acc: 0.9840
Epoch 4/5
33600/33600 [==============================] - 47s 1ms/step - loss: 0.0667 - acc: 0.9796 - val_loss: 0.0465 - val_acc: 0.9870
Epoch 5/5
33600/33600 [==============================] - 46s 1ms/step - loss: 0.0559 - acc: 0.9827 - val_loss: 0.0434 - val_acc: 0.9888
Test loss: 0.043413030635343776
Test

In [17]:
scores2 = model.predict_classes(test.values.reshape(test.shape[0], 28, 28, 1))

res = []
for i, p in enumerate(scores2):
    res.append([i + 1, p])   

result = pd.DataFrame(res, columns=['ImageId', 'Label'])
result.to_csv('model2.csv', index=None)